# Import

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import bz2
import urllib

data_folder = 'data/'

In [17]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/theopatron/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Creation of extended files

### FOR THE TIMELINE
keep quotation mentionning guns 

### FOR THE ANALYSIS
keep quotation mentionning guns <br/>
keep only those that have a speaker <br/>
keep american speakers <br/>
add colums corresponding to characteristics of speaker <br/>

In [3]:
lexical_field = ['guns', 'rifle', 'slaughter'] # To be continued

speakers = pd.read_parquet(data_folder + 'speaker_attributes.parquet')
label = pd.read_csv(data_folder + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

In [95]:
path_to_file = data_folder + 'quotes-2015.json.bz2' 
path_to_out = data_folder + 'quotes-2015-extended.json.bz2'

iter = 0

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            iter += 1
            if any(substring in instance['quotation'] for substring in lexical_field) and instance['qids'] != []: #FAIRE GAFFE NOM DE PAYS INCLUS
                speaker = speakers.loc[speakers['id'] == instance['qids'][0]].squeeze()
                
                if speaker.nationality is not None:
                    if any(substring == 'Q30' for substring in speaker.nationality):
                        instance['gender'] = 'male' if (speaker['gender'][0] == 'Q6581097') else 'female'
                        d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file
  
            #to test
            if iter > 200:
                break

True
True
True
True


# Analysis

In [14]:
def process_chunk(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    print(chunk.quotation[1])
    chunk['scores'] = chunk['quotation'].apply(lambda review: sid.polarity_scores(review))
    chunk['compound']  = chunk['scores'].apply(lambda score_dict: score_dict['compound'])
    for id, row in chunk.iterrows():
        print(chunk.quotation[id])
        print(chunk.compound[id])

with pd.read_json(data_folder + 'quotes-2015-extended.json.bz2', lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        process_chunk(chunk)
        break

Processing chunk with 4 rows
... through you -- the vibrant young athletes who are here today -- and through your hosts of modern Germany who have graciously welcomed you here, we have the opportunity to look forward, not backward.
' It is not now, nor has it ever been, the gold standard of trade agreements,' Sanders said. And he reached back to Clinton's 2002 vote to support the war in Iraq, an issue that plagued her eight years ago when she took the stage here. `When I came to that fork in the road I took the right road, even though it was not the popular road at the time,' he said.
-0.4822
... through you -- the vibrant young athletes who are here today -- and through your hosts of modern Germany who have graciously welcomed you here, we have the opportunity to look forward, not backward.
0.8979
I am incredibly pleased that my message is resonating with people all over the United States. There are so many American's that are ready to make our country great again, and I am the only o